In [90]:
import pickle, json, typing

import discovery as ds

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

import jax
import jax.numpy as jnp

In [2]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

/var/folders/4g/hbk3h5xd4xx3fhx604hn8d4h0000gn/T/ipykernel_60460/2920203772.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

Load pulsars

In [3]:
psrfiles = !ls ../data
psrs = [ds.Pulsar.read_feather(f'../data/{psrfile}') for psrfile in psrfiles[:2]]
psrs

Standard model 3a

In [4]:
Tspan = ds.getspan(psrs)

gbl = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                           for psr in psrs),
                          ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw'))

logl = gbl.logL

In [5]:
p0 = ds.sample_uniform(logl.params)
p0

{'B1855+09_rednoise_gamma': 5.6119111605995275,
 'B1855+09_rednoise_log10_A': -16.35949563559086,
 'B1937+21_rednoise_gamma': 0.5019082426723098,
 'B1937+21_rednoise_log10_A': -15.39642086534419,
 'gw_log10_A': -16.06931312018823,
 'gw_gamma': 5.7386216846992}

In [6]:
logl(p0)

Array(417438.47552066, dtype=float64)

Global GP with multiple components (different spectra and different ORFs)

In [7]:
Tspan = ds.getspan(psrs)

gbl2 = ds.GlobalLikelihood((ds.PulsarLikelihood([psr.residuals,
                                                ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                            for psr in psrs),
                           ds.makegp_fourier_global(psrs,
                                                    [ds.powerlaw, ds.powerlaw],
                                                    [ds.hd_orf, ds.dipole_orf],
                                                    14, T=Tspan, name='gw'))

logl2 = gbl2.logL

Note the parameter names follow from the orf functions

In [8]:
p2 = ds.sample_uniform(logl2.params)
p2

{'B1855+09_rednoise_gamma': 6.003178833888547,
 'B1855+09_rednoise_log10_A': -14.863933519478525,
 'B1937+21_rednoise_gamma': 5.905187126939223,
 'B1937+21_rednoise_log10_A': -15.578657396432844,
 'gw_dipoleorf_gamma': 6.669152449488989,
 'gw_dipoleorf_log10_A': -13.742421143117573,
 'gw_hdorf_gamma': 3.1335490892246476,
 'gw_hdorf_log10_A': -13.10715493864475}

In [9]:
logl2(p2)

Array(441190.13899692, dtype=float64)

Conditional distribution

In [16]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

In [17]:
psrfiles = !ls ../data
psrs = [ds.Pulsar.read_feather(f'../data/{psrfile}') for psrfile in psrfiles[:3]]
psrs

In [18]:
Tspan = ds.getspan(psrs)

outpsr = 0
outnoise = ds.makenoise_measurement_simple(psrs[0], {f'{psrs[outpsr].name}_efac': 1.0,
                                                     f'{psrs[outpsr].name}_log10_t2equad': 0.0})

gbl = ds.GlobalLikelihood([ds.PulsarLikelihood([psr.residuals,
                                                outnoise if ipsr == outpsr else ds.makenoise_measurement(psr, psr.noisedict),
                                                ds.makegp_ecorr(psr, psr.noisedict),
                                                ds.makegp_timing(psr, variance=1e-14),
                                                ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise')])
                           for ipsr, psr in enumerate(psrs)],
                          ds.makegp_fourier_global(psrs, ds.powerlaw, ds.hd_orf, 14, T=Tspan, name='gw'))

cond = gbl.conditional

In [19]:
cond.params

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A',
 'gw_log10_A',
 'gw_gamma']

In [31]:
p0 = ds.sample_uniform(cond.params)

In [32]:
cdist = cond(p0)

In [33]:
cdist[0][:28]

Array([ 4.46895733e-11,  7.86861565e-11, -5.66801649e-11,  5.21304143e-11,
       -1.58547713e-09,  1.16551804e-09, -8.63106967e-10,  4.07532032e-09,
        1.60643073e-09,  9.94700429e-09, -8.00332809e-10,  1.53170783e-08,
       -6.97962340e-09,  8.89280991e-09, -3.82158844e-09, -4.54742734e-09,
        3.48602561e-09, -5.16238232e-09, -1.33695262e-09,  1.93845692e-09,
       -8.83312174e-09,  1.37260854e-09, -7.19800093e-09, -1.13829452e-09,
       -3.45180319e-09, -3.99506457e-10, -2.15847754e-09, -1.68079123e-09],      dtype=float64)

In [34]:
cdist[0][28:56]

Array([ 1.09446407e-10,  2.02423197e-10, -5.02716224e-11,  1.14755624e-10,
       -2.50062314e-09,  1.80261778e-09, -1.38306632e-09,  6.43056380e-09,
        2.54262976e-09,  1.57006482e-08, -1.26224424e-09,  2.41800452e-08,
       -1.10183876e-08,  1.40329668e-08, -6.03644121e-09, -7.19169570e-09,
        5.49386446e-09, -8.13975195e-09, -2.10328150e-09,  3.07229938e-09,
       -1.39461048e-08,  2.15488758e-09, -1.13718762e-08, -1.78979239e-09,
       -5.43590088e-09, -6.27519453e-10, -3.40919393e-09, -2.65494274e-09],      dtype=float64)

Check that the prediction is insensitive to outpsr, but not to the others.

In [35]:
p1 = {**p0, f'{psrs[outpsr].name}_rednoise_gamma': 4.0, f'{psrs[outpsr].name}_rednoise_log10_A': -15.0}

In [36]:
cond(p1)[0][:28] - cdist[0][:28]

Array([-3.87740912e-26,  1.29246971e-26,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],      dtype=float64)

In [37]:
p2 = {**p0, f'{psrs[outpsr+1].name}_rednoise_gamma': 4.0, f'{psrs[outpsr+1].name}_rednoise_log10_A': -15.0}

In [38]:
cond(p2)[0][:28] - cdist[0][:28]

Array([ 8.49072814e-07,  1.66280476e-06,  5.28379853e-07,  4.57972398e-07,
        3.78560753e-08, -3.35389694e-07, -1.36224202e-07, -5.18684020e-08,
        1.03362150e-08, -5.02732445e-09,  2.58353365e-08,  1.33474750e-08,
       -2.34281863e-09, -4.88793770e-08, -2.89311648e-08, -2.20320386e-08,
       -3.71197075e-09,  2.90077884e-08,  5.51192465e-09,  1.26535775e-08,
       -1.08733561e-08, -8.16423541e-09, -1.36996601e-09,  6.76778844e-09,
        9.78727948e-09,  1.55570211e-13,  3.46148374e-09,  1.57174236e-08],      dtype=float64)

The second term in the conditional is `cf = L^-1` where S = L L^T is the variance of the normal distribution.

In [39]:
outrange  = slice(outpsr*28, (outpsr+1)*28)

mean      = cdist[0][outrange]
righthand = jnp.identity(len(cdist[0]))[:, outrange]
sigmalow  = ds.matrix.jsp.linalg.cho_solve(cdist[1], righthand)
variance  = sigmalow[outrange, :]

Check that the mean matches. Note this requires that a list be fed to ds.GlobalLikelihood above, but it can be safely changed to a generator in production.

In [40]:
ksolves = [psl.N.make_kernelsolve(psl.y, F) for psl, F in zip(gbl.psls, gbl.globalgp.Fs)]
FtNmy = ds.matrix.jnp.concatenate([ksolve(p0)[0] for ksolve in ksolves])
ds.matrix.jnp.dot(sigmalow.T, FtNmy) - cdist[0][:28]

Array([-1.93870456e-26, -1.29246971e-26,  5.16987883e-26, -6.46234854e-27,
       -8.27180613e-25,  0.00000000e+00,  5.16987883e-25, -8.27180613e-25,
       -1.24077092e-24, -8.27180613e-24, -4.13590306e-25,  3.30872245e-24,
       -4.96308368e-24, -3.30872245e-24, -2.48154184e-24, -1.65436123e-24,
        2.06795153e-24,  3.30872245e-24,  2.06795153e-25,  8.27180613e-25,
        3.30872245e-24,  2.06795153e-25,  0.00000000e+00,  2.06795153e-25,
        0.00000000e+00, -1.03397577e-25,  0.00000000e+00,  8.27180613e-25],      dtype=float64)

In [83]:
import imp

imp.reload(ds.matrix)
imp.reload(ds.signals)
imp.reload(ds.likelihood)
imp.reload(ds.prior)
imp.reload(ds.pulsar)
imp.reload(ds)

<module 'discovery' from '/Users/vallis/Documents/discovery/src/discovery/__init__.py'>

In [84]:
psr = psrs[outpsr]
spl = ds.PulsarLikelihood([psr.residuals,
                           ds.makenoise_measurement(psr, psr.noisedict),
                           ds.makegp_ecorr(psr, psr.noisedict),
                           ds.makegp_timing(psr, variance=1e-14),
                           ds.makegp_fourier(psr, ds.powerlaw, 30, T=Tspan, name='rednoise'),
                           ds.makegp_fourier_delay(psr, 14, T=Tspan, name='gw'),
                           ds.makegp_fourier_variance(psr, 14, T=Tspan, name='gw')
                           ])

logl = spl.logL

In [85]:
logl.params

['B1855+09_gw_mean(28)',
 'B1855+09_gw_variance(28,28)',
 'B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A']

Check

In [511]:
Nmat = spl.N.P_var.getN({**pout, 'B1855+09_gw_delay': mean, 'B1855+09_gw_variance': variance})
print(np.sum(Nmat[:60,:60] - np.diag(np.diag(Nmat[:60,:60]))))
print(Nmat[60:,60:])

0.0
[[ 1.31494579e-14  4.00657769e-18 -2.98759762e-17 -2.79209532e-17
   1.29008727e-18  1.01109141e-17  6.00787470e-19  5.59450723e-18
  -4.55038036e-19  1.08262617e-18  6.30991243e-18 -3.21841822e-18
  -3.23834052e-18 -3.51155943e-18 -4.35746231e-18  4.11263501e-18
   3.35445373e-18 -1.81548818e-19 -5.77642544e-19 -2.96220468e-18
  -3.03322602e-19  3.08492645e-19  3.76698741e-19  3.75129948e-19
  -7.41820667e-19  4.88516686e-19  2.62218092e-19 -2.99060222e-20]
 [ 4.00657769e-18  1.31587101e-14  1.86841513e-17 -1.57723352e-17
   2.75108707e-18 -1.74774799e-18  1.69474047e-18  7.00973434e-18
   9.77371427e-18 -2.37697012e-18  1.39459387e-19 -2.71489306e-18
   4.44432547e-18  2.16579908e-18  2.26171643e-18 -3.68347619e-18
  -2.79487602e-20  2.26300804e-19  9.55471234e-19  1.73020885e-19
  -1.00805657e-18 -1.73573487e-18 -1.66181940e-18  1.11403682e-18
   8.51063217e-19  3.69204209e-19 -3.75578525e-19 -3.59951012e-19]
 [-2.98759762e-17  1.86841513e-17  3.59608561e-15  8.36058937e-18
  -1

In [86]:
pout = ds.sample_uniform([p for p in logl.params if 'gw' not in p])

In [87]:
logl({**pout, 'B1855+09_gw_mean(28)': mean, 'B1855+09_gw_variance(28,28)': variance})

Array(100658.6504483, dtype=float64)

In [91]:
chain = pd.read_feather('../data/NG15yr-m3a-chain.feather')

In [ ]:
nsamples = 10

In [ ]:
for i in range(nsamples):
    

In [103]:
p = chain.sample().iloc[0].to_dict()

In [107]:
p

{'B1855+09_red_noise_gamma': 5.338995456695557,
 'B1855+09_red_noise_log10_A': -14.674732208251953,
 'B1937+21_red_noise_gamma': 4.469628810882568,
 'B1937+21_red_noise_log10_A': -13.571699142456055,
 'B1953+29_red_noise_gamma': 2.5103724002838135,
 'B1953+29_red_noise_log10_A': -12.703226089477539,
 'J0023+0923_red_noise_gamma': 4.046350479125977,
 'J0023+0923_red_noise_log10_A': -18.141340255737305,
 'J0030+0451_red_noise_gamma': 5.45987606048584,
 'J0030+0451_red_noise_log10_A': -14.620365142822266,
 'J0340+4130_red_noise_gamma': 2.3450794219970703,
 'J0340+4130_red_noise_log10_A': -16.415000915527344,
 'J0406+3039_red_noise_gamma': 1.9178773164749146,
 'J0406+3039_red_noise_log10_A': -14.650755882263184,
 'J0437-4715_red_noise_gamma': 0.5288191437721252,
 'J0437-4715_red_noise_log10_A': -16.663171768188477,
 'J0509+0856_red_noise_gamma': 3.2450449466705322,
 'J0509+0856_red_noise_log10_A': -19.808359146118164,
 'J0557+1551_red_noise_gamma': 1.8561710119247437,
 'J0557+1551_red_nois

In [106]:
[par for par in cond.params if par not in p]

['B1855+09_rednoise_gamma',
 'B1855+09_rednoise_log10_A',
 'B1937+21_rednoise_gamma',
 'B1937+21_rednoise_log10_A',
 'B1953+29_rednoise_gamma',
 'B1953+29_rednoise_log10_A']